<a href="https://colab.research.google.com/github/Akinori2/Alocacao_de_laboratorios-SA-/blob/main/Aloca%C3%A7%C3%A3o_de_Laboratorios(SA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bibliotecas usadas

In [1129]:
import random
import math
from collections import defaultdict

Dados das matérias padronizados

In [1130]:
materias = {
    # Importancia 3 - 8 matérias
    'Programação 1': {'alunos': 60, 'pratica': True, 'importancia': 3, 'periodo': 1},
    'Algoritmos e Complexidade': {'alunos': 50, 'pratica': True, 'importancia': 3, 'periodo': 2},
    'Estruturas de Dados': {'alunos': 50, 'pratica': True, 'importancia': 3, 'periodo': 3},
    'Banco de Dados': {'alunos': 40, 'pratica': True, 'importancia': 3, 'periodo': 4},
    'Redes de Computadores': {'alunos': 30, 'pratica': True, 'importancia': 3, 'periodo': 5},
    'Inteligência Artificial': {'alunos': 20, 'pratica': True, 'importancia': 3, 'periodo': 6},
    'Desenvolvimento de Software': {'alunos': 30, 'pratica': True, 'importancia': 3, 'periodo': 7},
    'Projeto de TCC': {'alunos': 10, 'pratica': True, 'importancia': 3, 'periodo': 8},

    # Importancia 2 - 8 matérias
    'Lógica para Computação': {'alunos': 50, 'pratica': False, 'importancia': 2, 'periodo': 1},
    'Arquitetura de Computadores': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 2},
    'Sistemas Operacionais': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 3},
    'Engenharia de Software': {'alunos': 40, 'pratica': True, 'importancia': 2, 'periodo': 4},
    'Segurança da Informação': {'alunos': 30, 'pratica': True, 'importancia': 2, 'periodo': 5},
    'Computação em Nuvem': {'alunos': 25, 'pratica': True, 'importancia': 2, 'periodo': 6},
    'Computação Gráfica': {'alunos': 20, 'pratica': True, 'importancia': 2, 'periodo': 7},
    'Projeto Integrador': {'alunos': 15, 'pratica': True, 'importancia': 2, 'periodo': 8},

    # Importancia 1 - 8 matérias
    'Introdução à Computação': {'alunos': 60, 'pratica': False, 'importancia': 1, 'periodo': 1},
    'Matemática Discreta': {'alunos': 50, 'pratica': False, 'importancia': 1, 'periodo': 2},
    'Álgebra Linear': {'alunos': 40, 'pratica': False, 'importancia': 1, 'periodo': 3},
    'Cálculo Numérico': {'alunos': 35, 'pratica': False, 'importancia': 1, 'periodo': 4},
    'Teoria da Computação': {'alunos': 30, 'pratica': False, 'importancia': 1, 'periodo': 5},
    'Sistemas Digitais': {'alunos': 30, 'pratica': True, 'importancia': 1, 'periodo': 6},
    'Ética na Computação': {'alunos': 40, 'pratica': False, 'importancia': 1, 'periodo': 7},
    'Empreendedorismo em TI': {'alunos': 35, 'pratica': False, 'importancia': 1, 'periodo': 8}
}


salas disponiveis

In [1131]:
salas = {
    'Sala Grande': {'limite': 60, 'tipo': 'normal'},
    'Sala Pequena': {'limite': 25, 'tipo': 'especial'}
}

horarios disponiveis

In [1132]:
horarios = ['Manhã (8-10)', 'Tarde (10-12)']
dias_semana = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']

Divide as matérias priorizando importância maior na sala pequena

In [1133]:
def separar_materias():
    materias_grande = []
    materias_pequena = []

    for nome, info in materias.items():
        if not info['pratica']:
            continue

        # Prioriza alocar matérias importantes na sala pequena
        if info['alunos'] <= salas['Sala Pequena']['limite']:
            if info['importancia'] >= 2:  # Matérias de média/alta importância
                materias_pequena.append(nome)
            elif info['importancia'] == 1 and info['alunos'] > salas['Sala Grande']['limite']:
                materias_pequena.append(nome)
        elif info['alunos'] <= salas['Sala Grande']['limite']:
            materias_grande.append(nome)

    return materias_grande, materias_pequena

Garante matérias suficientes sem repetir

In [1134]:
def completar_horarios(lista_materias, sala):
    total_needed = len(dias_semana) * len(horarios)
    disponiveis = lista_materias.copy()

    # Pega matérias não usadas que se encaixam
    for nome, info in materias.items():
        if (nome not in disponiveis and info['pratica'] and
            info['alunos'] <= salas[sala]['limite']):
            disponiveis.append(nome)
            if len(disponiveis) >= total_needed:
                break

    # Se ainda faltar, pega as que se encaixam mesmo não sendo ideais
    if len(disponiveis) < total_needed:
        for nome, info in materias.items():
            if nome not in disponiveis and info['pratica']:
                disponiveis.append(nome)
                if len(disponiveis) >= total_needed:
                    break

    return disponiveis[:total_needed]

Cria grade inicial válida

In [1135]:
def criar_grade_inicial():
    materias_g, materias_p = separar_materias()
    materias_g = completar_horarios(materias_g, 'Sala Grande')
    materias_p = completar_horarios(materias_p, 'Sala Pequena')

    random.shuffle(materias_g)
    random.shuffle(materias_p)

    grade = {}
    for dia in dias_semana:
        for horario in horarios:
            if materias_g:
                grade[('Sala Grande', dia, horario)] = materias_g.pop()
            if materias_p:
                grade[('Sala Pequena', dia, horario)] = materias_p.pop()

    return grade

Calcula qualidade com forte penalidade se importância média da pequena não for maior

In [1136]:
def avaliar_grade(grade):
    """Calcula qualidade com forte penalidade se importância média da pequena não for maior"""
    pontos = 0
    contagem = defaultdict(int)
    soma_importancia = {'Sala Grande': 0, 'Sala Pequena': 0}
    contagem_salas = {'Sala Grande': 0, 'Sala Pequena': 0}

    for (sala, _, _), materia in grade.items():
        info = materias[materia]
        contagem[materia] += 1
        soma_importancia[sala] += info['importancia']
        contagem_salas[sala] += 1

        # Verifica regras básicas
        if info['alunos'] > salas[sala]['limite']:
            pontos += 30
        if contagem[materia] > 1:
            pontos += 50

    # Calcula médias
    media_grande = soma_importancia['Sala Grande'] / max(1, contagem_salas['Sala Grande'])
    media_pequena = soma_importancia['Sala Pequena'] / max(1, contagem_salas['Sala Pequena'])

    # Penalidade pesada se a média da pequena não for maior
    if media_pequena <= media_grande:
        pontos += 100 * (media_grande - media_pequena + 0.5)

    return pontos

Cria pequena mudança na grade

In [1137]:
def gerar_variacao(grade):
    nova_grade = grade.copy()
    chaves = list(nova_grade.keys())

    if len(chaves) >= 2:
        i, j = random.sample(range(len(chaves)), 2)
        chave1, chave2 = chaves[i], chaves[j]
        nova_grade[chave1], nova_grade[chave2] = nova_grade[chave2], nova_grade[chave1]

    return nova_grade

Otimiza a grade com foco na diferença de importância

In [1138]:
def melhorar_grade():
    melhor = criar_grade_inicial()
    melhor_pontos = avaliar_grade(melhor)
    temp = 1000

    for passo in range(1, 15001):  # Aumentei as iterações
        nova = gerar_variacao(melhor)
        pontos = avaliar_grade(nova)

        if pontos < melhor_pontos or random.random() < math.exp((melhor_pontos - pontos)/temp):
            melhor, melhor_pontos = nova, pontos

        temp *= 0.999  # Resfriamento mais lento

        if passo % 2000 == 0:
            # Mostra estatísticas parciais
            sg = [m for (s,_,_), m in melhor.items() if s == 'Sala Grande']
            sp = [m for (s,_,_), m in melhor.items() if s == 'Sala Pequena']
            mg = sum(materias[m]['importancia'] for m in sg)/len(sg) if sg else 0
            mp = sum(materias[m]['importancia'] for m in sp)/len(sp) if sp else 0
            print(f"Passo {passo}: Pontos={melhor_pontos} | Médias: Grande={mg:.2f}, Pequena={mp:.2f}")

    return melhor

Exibe a grade formatada

In [1139]:
def mostrar_resultado(grade):
    """Exibe a grade formatada"""
    for sala in salas:
        print(f"\n{sala} ({salas[sala]['limite']} alunos)")
        print("-" * 50)
        print(f"{'Dia':<10} | {'Manhã':<20} | {'Tarde':<20}")
        print("-" * 50)

        for dia in dias_semana:
            linha = f"{dia:<10} | "
            for horario in horarios:
                materia = grade.get((sala, dia, horario), "VAZIO")
                info = materias.get(materia, {})

                if sala == 'Sala Grande' and info.get('importancia', 0) > 1:
                    materia = f"*{materia}*"
                elif sala == 'Sala Pequena' and info.get('importancia', 0) < 2:
                    materia = f"*{materia}*"

                linha += f"{materia:<20} | "
            print(linha)
        print("-" * 50)

Execução

In [1140]:
print("Otimizando grade de horários...")
grade_otimizada = melhorar_grade()

print("\nResultado final:")
mostrar_resultado(grade_otimizada)

Passo 2000: Pontos=490.0 | Médias: Grande=2.70, Pequena=2.60
Passo 4000: Pontos=430 | Médias: Grande=2.60, Pequena=2.70
Passo 6000: Pontos=400 | Médias: Grande=2.60, Pequena=2.70
Passo 8000: Pontos=400 | Médias: Grande=2.60, Pequena=2.70
Passo 10000: Pontos=400 | Médias: Grande=2.60, Pequena=2.70
Passo 12000: Pontos=400 | Médias: Grande=2.60, Pequena=2.70
Passo 14000: Pontos=400 | Médias: Grande=2.60, Pequena=2.70

Resultado final:

Sala Grande (60 alunos)
--------------------------------------------------
Dia        | Manhã                | Tarde               
--------------------------------------------------
Segunda    | *Banco de Dados*     | *Sistemas Operacionais* | 
Terça      | *Algoritmos e Complexidade* | *Redes de Computadores* | 
Quarta     | *Arquitetura de Computadores* | *Programação 1*      | 
Quinta     | *Segurança da Informação* | *Engenharia de Software* | 
Sexta      | *Algoritmos e Complexidade* | *Redes de Computadores* | 
---------------------------------------

Estatísticas

In [1141]:
print("\nEstatísticas:")
for sala in salas:
    aulas = [m for (s,_,_), m in grade_otimizada.items() if s == sala]
    print(f"\n{sala}:")
    print(f"- Aulas: {len(aulas)}")
    print(f"- Matérias únicas: {len(set(aulas))}")
    print(f"- Importância média: {sum(materias[m]['importancia'] for m in aulas)/len(aulas):.1f}")


Estatísticas:

Sala Grande:
- Aulas: 10
- Matérias únicas: 8
- Importância média: 2.6

Sala Pequena:
- Aulas: 10
- Matérias únicas: 9
- Importância média: 2.7
